In [7]:
!pip install --quiet gin-config
!pip install --quiet tensorflow-hub
!pip install --quiet tensorflow-addons
!pip install --quiet sentencepiece
!pip install --quiet pycocotools
!pip install --quiet opencv-python-headless
!pip install --quiet seqeval
!pip install --quiet tensorflow_text
!pip install --quiet sacrebleu

In [17]:
import sys, re, math, time, os
sys.path.append('../../../models') # Path to Tensorflow model garden installation directory
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import matplotlib as mpl
import pprint as pp
AUTO = tf.data.AUTOTUNE
print("Tensorflow version", tf.__version__)

# Tensorflow Model Garden imports
from official.core import task_factory
from official.core import exp_factory
from official.common import registry_imports
from official.core import config_definitions as cfg
from official.modeling import optimization
from official.vision.beta.tasks import retinanet as retinanet_task
from official.vision.beta.configs import retinanet as retinanet_cfg
from official.vision.beta.configs import backbones as backbones_cfg
from official.modeling.hyperparams import params_dict as params_dict
from official.core import train_lib
from official.vision.beta.serving import export_saved_model_lib

# for debugging
from official.vision.beta.ops import preprocess_ops
from official.vision.beta.ops import box_ops

import official as modelgarden

#from official.vision.detection.configs import factory as tf_garden_config_factory
#from official.modeling.hyperparams import params_dict as tf_garden_params_dict
#from official.vision.detection.modeling import factory as tf_garden_model_factory
#from official.vision.detection.dataloader import input_reader as tf_garden_input_reader
#from official.vision.detection.dataloader import mode_keys as tf_garden_mode_keys
#from official.vision.detection.executor.detection_executor import DetectionDistributedExecutor as tf_garden_training_loop

Tensorflow version 2.5.0


# TPU / GPU detection 

In [2]:
strategy = tf.distribute.get_strategy()

try: # detect TPUs
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
    strategy = tf.distribute.TPUStrategy(tpu)
    #policy = tf.keras.mixed_precision.experimental.Policy('mixed_bfloat16')
    #tf.keras.mixed_precision.experimental.set_policy(policy)
except ValueError: # detect GPUs or multi-GPU machines
    strategy = tf.distribute.MirroredStrategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

INFO:tensorflow:Initializing the TPU system: martin-tpuv3-8-tf25


INFO:tensorflow:Initializing the TPU system: martin-tpuv3-8-tf25


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


REPLICAS:  8


# Configuration

In [3]:
TRAIN_DATA_PATH_PATTERN = 'gs://practical-ml-vision-book/arthropod_detection_tfr/size_w1024px/*.train.tfrec'
VALID_DATA_PATH_PATTERN = 'gs://practical-ml-vision-book/arthropod_detection_tfr/size_w1024px/*.test.tfrec'
#SPINET_MOBILE_CHECKPOINT = '../../../ckpt/'
SPINET_MOBILE_CHECKPOINT = 'gs://practical-ml-vision-book/arthropod_detection_tfr/spinenet_mobile_checkpoint/'

BATCH_SIZE = 32 * strategy.num_replicas_in_sync

EPOCHS = 80

MODEL_DIR = 'gs://ml1-demo-martin/arthropod_jobs/job' + str(int(time.time()))
#MODEL_DIR = "gs://ml1-demo-martin/arthropod_jobs/job1624685364"

RAW_CLASSES = ['Lepidoptera', 'Hymenoptera', 'Hemiptera', 'Odonata', 'Diptera', 'Araneae', 'Coleoptera',
               '_truncated', '_blurred', '_occluded', ]
CLASSES = [klass for klass in RAW_CLASSES if klass not in ['_truncated', '_blurred', '_occluded']]

#PAD_MAX_BOXES = 50

# Lepidoptera = butterfies and moths
# Hymenoptera = wasps, bees and ants
# Hemiptera = true bugs (cicadas, aphids, shield bugs, ...)
# Odonata = dragonflies
# Diptera = fies
# Araneae = spiders
# Coleoptera = beetles

# NOT IN DATASET
# Orthoptera = grasshoppers

def count_data_items(filenames):
    # the number of data items is written in the name of the .tfrec files, i.e. flowers00-230.tfrec = 230 data items
    n = [int(re.compile(r"-([0-9]*)\.").search(filename).group(1)) for filename in filenames]
    return int(np.sum(n))

TRAIN_FILENAMES = tf.io.gfile.glob(TRAIN_DATA_PATH_PATTERN)
VALID_FILENAMES = tf.io.gfile.glob(VALID_DATA_PATH_PATTERN)

print("Training dataset:")
print(f"Found {len(TRAIN_FILENAMES)} TFRecord files.")
NB_TRAIN_IMAGES = count_data_items(TRAIN_FILENAMES)
STEPS_PER_EPOCH = NB_TRAIN_IMAGES // BATCH_SIZE
print(f"{NB_TRAIN_IMAGES} images")
print("Steps per epoch:", STEPS_PER_EPOCH)
print()
print("Validation dataset:")
print(f"Found {len(VALID_FILENAMES)} TFRecord files.")
NB_VALID_IMAGES = count_data_items(VALID_FILENAMES)
VALID_STEPS = NB_VALID_IMAGES // BATCH_SIZE
print(f"{NB_VALID_IMAGES} images")
print("Validation steps:", VALID_STEPS)
print()
print("Global batch size:", BATCH_SIZE)
print("Model dir:", MODEL_DIR)

Training dataset:
Found 24 TFRecord files.
11544 images
Steps per epoch: 45

Validation dataset:
Found 8 TFRecord files.
3832 images
Validation steps: 14

Global batch size: 256
Model dir: gs://ml1-demo-martin/arthropod_jobs/job1625758868


# Visualization utilities

In [4]:
def no_decorations(ax):
    ax.axes.get_xaxis().set_visible(False)
    ax.axes.get_yaxis().set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.spines['left'].set_visible(False)
    

def display_detections(images, offsets, resizes, detections, classnames, ground_truth_boxes=[]):
    # scale and offset the detected boxes back to original image coordinates
    boxes   = [[ (x,y,w,h)  for _, x, y, w, h, score, klass in detection_list] for detection_list in detections]
    boxes   = [[ (x-ofs[1], y-ofs[0], w, h) for x,y,w,h in boxlist ] for boxlist, ofs in zip(boxes, offsets)]
    boxes   = [[ (x*rsz, y*rsz, w*rsz, h*rsz) for x,y,w,h in boxlist ] for boxlist, rsz in zip(boxes, resizes)]
    classes = [[ int(klass) for _, x, y, w, h, score, klass in detection_list] for detection_list in detections]
    scores  = [[ score      for _, x, y, w, h, score, klass in detection_list] for detection_list in detections]
    display_with_boxes(images, boxes, classes, scores, classnames, ground_truth_boxes)
    
    
# images, boxes and classes must have the same number of elements
# scores can be en empty list []. If it is not empty, it must also
# have the same number of elements.
# classnames is the list of possible classes (strings)
def display_with_boxes(images, boxes, classes, scores, classnames, ground_truth_boxes=[]):
    N = len(images)
    sqrtN = int(math.ceil(math.sqrt(N)))
    aspect = sum([im.shape[1]/im.shape[0] for im in images])/len(images) # mean aspect ratio of images
    fig = plt.figure(figsize=(15,15/aspect), frameon=False)
    
    for k in range(N):
        ax = plt.subplot(sqrtN, sqrtN, k+1)
        no_decorations(ax)
        plt.imshow(images[k])
        
        if ground_truth_boxes:
            for box in ground_truth_boxes[k]:
                x, y, w, h = (box[0], box[1], box[2]-box[0], box[3]-box[1]) # convert x1 y1 x2 y2 into xywh
                #x, y, w, h = (box[0], box[1], box[2], box[3])
                rect = mpl.patches.Rectangle((x, y),w,h,linewidth=4,edgecolor='#FFFFFFA0',facecolor='none')
                ax.add_patch(rect)

        for i, (box, klass) in enumerate(zip(boxes[k], classes[k])):
            x, y, w, h = (box[0], box[1], box[2]-box[0], box[3]-box[1]) # convert x1 y1 x2 y2 into xywh
            #x, y, w, h = (box[0], box[1], box[2], box[3])
            #label = classnames[klass-1] # predicted classes are 1-based
            label = classnames[klass]
            if scores:
                label += ' ' + str(int(scores[k][i]*100)) + '%' 
            rect = mpl.patches.Rectangle((x, y),w,h,linewidth=4,edgecolor='#00000080',facecolor='none')
            ax.add_patch(rect)
            rect = mpl.patches.Rectangle((x, y),w,h,linewidth=2,edgecolor='#FFFF00FF',facecolor='none')
            ax.add_patch(rect)
            plt.text(x, y, label, size=16, ha="left", va="top", color='#FFFF00FF',
                     bbox=dict(boxstyle="round", ec='#00000080', fc='#0000004E', linewidth=3) )
            plt.text(x, y, label, size=16, ha="left", va="top", color='#FFFF00FF',
                     bbox=dict(boxstyle="round", ec='#FFFF00FF', fc='#0000004E', linewidth=1.5) )
    plt.tight_layout()
    plt.subplots_adjust(wspace=0.02, hspace=0.02)
    plt.show()

# Model

In [5]:
OUTPUT_SIZE = [384, 384]
#params = exp_factory.get_exp_config('retinanet_resnetfpn_coco')
params = exp_factory.get_exp_config('retinanet')
params.task.model.num_classes = len(CLASSES)+1
params.task.model.input_size = [*OUTPUT_SIZE, 3] # this automatically configures the input reader to random crop training images ?
#pp.pprint(params.as_dict())

In [6]:
train_data_cfg=retinanet_cfg.DataConfig(
    input_path=TRAIN_DATA_PATH_PATTERN,
    is_training=True,
    global_batch_size=BATCH_SIZE,
    parser=retinanet_cfg.Parser(aug_rand_hflip=True, aug_scale_min=0.7, aug_scale_max=2.0))

valid_data_cfg=retinanet_cfg.DataConfig(
    input_path=VALID_DATA_PATH_PATTERN,
    is_training=False,
    global_batch_size=BATCH_SIZE)

trainer_cfg=cfg.TrainerConfig(
    train_steps=EPOCHS * STEPS_PER_EPOCH,
    validation_steps=VALID_STEPS,
    validation_interval=8*STEPS_PER_EPOCH,
    steps_per_loop=STEPS_PER_EPOCH,
    summary_interval=STEPS_PER_EPOCH,
    checkpoint_interval=8*STEPS_PER_EPOCH)

MUL=3
optim_cfg = optimization.OptimizationConfig({
    'optimizer': {
                  'type': 'sgd',
                  'sgd': {'momentum': 0.9}},
                  #'type': 'adam'},
    #'learning_rate': {'type': 'exponential',
    #                  'exponential': {'initial_learning_rate': 0.001 * strategy.num_replicas_in_sync,
    #                                  'decay_steps':15,
    #                                  'decay_rate':0.96,
    #                                  'offset': 0}
    #                 },
    'learning_rate': {'type': 'stepwise',
                      'stepwise': {'boundaries': [MUL * 5 * STEPS_PER_EPOCH,
                                                  MUL * 10 * STEPS_PER_EPOCH,
                                                  MUL * 15 * STEPS_PER_EPOCH,
                                                  MUL * 20 * STEPS_PER_EPOCH,
                                                  MUL * 25 * STEPS_PER_EPOCH],
                                   'values': [0.016, #0.01,
                                              0.008, #0.005,
                                              0.004, #0.0025,
                                              0.002, #0.001,
                                              0.001, #0.0005,
                                              0.0005]} #0.00025]}
                     },
    #'warmup': {'type': 'linear','linear': {'warmup_steps': 5*STEPS_PER_EPOCH, 'warmup_learning_rate': 0.00001}}
})

params = params_dict.override_params_dict(params, {'task': {'train_data': train_data_cfg.as_dict(),
                                                           'validation_data': valid_data_cfg.as_dict()}}, is_strict=True)
trainer_cfg = params_dict.override_params_dict(trainer_cfg, {'optimizer_config': optim_cfg.as_dict()}, is_strict=True)
params = params_dict.override_params_dict(params, {'trainer': trainer_cfg.as_dict()}, is_strict=True)
#RESNET50_CHECKPOINT = 'https://storage.cloud.google.com/cloud-tpu-checkpoints/model-garden-vision/detection/resnet50-2018-02-07.tar.gz'
#RESNET50_CHECKPOINT = 'gs://cloud-tpu-checkpoints/retinanet/resnet50-checkpoint-2018-02-07'
#RESNET50_CHECKPOINT = 'gs://cloud-tpu-checkpoints/retinanet/resnet50-checkpoint-2018-02-07'
#RESNET50_CHECKPOINT = 'gs://cloud-tpu-checkpoints/model-garden-vision/detection/resnet50-2018-02-07.tar.gz'
#RESNET50_CHECKPOINT = 'gs://cloud-tpu-checkpoints/resnet/resnet50/'
#RESNET50_CHECKPOINT = 'gs://cloud-tpu-checkpoints/detection/retinanet/r50-fpn.tar.gz'
backbone_config = backbones_cfg.Backbone(type='spinenet_mobile', spinenet_mobile=backbones_cfg.SpineNetMobile())
params = params_dict.override_params_dict(params, {'task': {'model': {'backbone': backbone_config.as_dict()}}}, is_strict=True)
params = params_dict.override_params_dict(params, {'task': {'init_checkpoint': SPINET_MOBILE_CHECKPOINT}}, is_strict=True)
params = params_dict.override_params_dict(params, {'task': {'init_checkpoint_modules': 'backbone'}}, is_strict=True)

# this works as well:
# params.task.train_data = train_data
#params.task.validation_data = None
print("-------------------------------------")
pp.pprint(params.as_dict())

-------------------------------------
{'runtime': {'all_reduce_alg': None,
             'batchnorm_spatial_persistent': False,
             'dataset_num_private_threads': None,
             'default_shard_dim': -1,
             'distribution_strategy': 'mirrored',
             'enable_xla': False,
             'gpu_thread_mode': None,
             'loss_scale': None,
             'mixed_precision_dtype': None,
             'num_cores_per_replica': 1,
             'num_gpus': 0,
             'num_packs': 1,
             'per_gpu_thread_count': 0,
             'run_eagerly': False,
             'task_index': -1,
             'tpu': None,
             'tpu_enable_xla_dynamic_padder': None,
             'worker_hosts': None},
 'task': {'annotation_file': None,
          'init_checkpoint': 'gs://practical-ml-vision-book/arthropod_detection_tfr/spinenet_mobile_checkpoint/',
          'init_checkpoint_modules': 'backbone',
          'losses': {'box_loss_weight': 50,
                     'foca

In [7]:
#with strategy.scope():
task = task_factory.get_task(params.task, logging_dir=MODEL_DIR)
    # this works too:
    # task = retinanet_task.RetinaNetTask(params.task)
#task.build_model()

In [8]:
pp.pprint(train_data_cfg.as_dict())
pp.pprint(valid_data_cfg.as_dict())

{'block_length': 1,
 'cache': False,
 'cycle_length': None,
 'decoder': {'simple_decoder': {'regenerate_source_id': False},
             'type': 'simple_decoder'},
 'deterministic': None,
 'drop_remainder': True,
 'dtype': 'bfloat16',
 'enable_tf_data_service': False,
 'file_type': 'tfrecord',
 'global_batch_size': 256,
 'input_path': 'gs://practical-ml-vision-book/arthropod_detection_tfr/size_w1024px/*.train.tfrec',
 'is_training': True,
 'parser': {'aug_rand_hflip': True,
            'aug_scale_max': 2.0,
            'aug_scale_min': 0.7,
            'match_threshold': 0.5,
            'max_num_instances': 100,
            'num_channels': 3,
            'skip_crowd_during_training': True,
            'unmatched_threshold': 0.5},
 'seed': None,
 'sharding': True,
 'shuffle_buffer_size': 10000,
 'tf_data_service_address': None,
 'tf_data_service_job_name': None,
 'tfds_as_supervised': False,
 'tfds_data_dir': '',
 'tfds_name': '',
 'tfds_skip_decoding_feature': '',
 'tfds_split': ''}
{

In [9]:
#DEBUG
filenames = tf.io.gfile.glob(TRAIN_DATA_PATH_PATTERN)
ds = tf.data.TFRecordDataset(filenames)
dec = modelgarden.vision.beta.dataloaders.tf_example_decoder.TfExampleDecoder()
ds = ds.map(dec.decode)
print(ds)

<MapDataset shapes: {source_id: (), image: (None, None, 3), height: (), width: (), groundtruth_classes: (None,), groundtruth_is_crowd: (None,), groundtruth_area: (None,), groundtruth_boxes: (None, 4)}, types: {source_id: tf.string, image: tf.uint8, height: tf.int64, width: tf.int64, groundtruth_classes: tf.int64, groundtruth_is_crowd: tf.bool, groundtruth_area: tf.float32, groundtruth_boxes: tf.float32}>


In [11]:
#DEBUG
def _parse_train_data(data):
    """Parses data for training and evaluation."""
    classes = data['groundtruth_classes']
    boxes = data['groundtruth_boxes']
    is_crowds = data['groundtruth_is_crowd']

    # Gets original image and its size.
    image = data['image']

    image_shape = tf.shape(input=image)[0:2]

    # Normalizes image with mean and std pixel values.
    image = preprocess_ops.normalize_image(image)

    # Flips image randomly during training.
    image, boxes, _ = preprocess_ops.random_horizontal_flip(image, boxes)

    # Converts boxes from normalized coordinates to pixel coordinates.
    boxes = box_ops.denormalize_boxes(boxes, image_shape)

    OUTPUT_SIZE = 4
    MAX_LEVEL = 7
    AUG_SCALE_MIN = 0.7
    AUG_SCALE_MAX = 2.0
    # Resizes and crops image.
#    image, image_info = preprocess_ops.resize_and_crop_image(
#        image,
#        OUTPUT_SIZE,
#        padded_size=preprocess_ops.compute_padded_size(OUTPUT_SIZE,
#                                                       2**MAX_LEVEL),
#        aug_scale_min=AUG_SCALE_MIN,
#        aug_scale_max=AUG_SCALE_MAX)
#    image_height, image_width, _ = image.get_shape().as_list()

    # Resizes and crops boxes.
#    image_scale = image_info[2, :]
#    offset = image_info[3, :]
#    boxes = preprocess_ops.resize_and_crop_boxes(boxes, image_scale,
#                                                 image_info[1, :], offset)
    # Filters out ground truth boxes that are all zeros.
    indices = box_ops.get_non_empty_box_indices(boxes)
    boxes = tf.gather(boxes, indices)
    classes = tf.gather(classes, indices)

#    # Assigns anchors.
#    input_anchor = anchor.build_anchor_generator(
#        min_level=self._min_level,
#        max_level=self._max_level,
#        num_scales=self._num_scales,
#        aspect_ratios=self._aspect_ratios,
#        anchor_size=self._anchor_size)
#    anchor_boxes = input_anchor(image_size=(image_height, image_width))
#    anchor_labeler = anchor.AnchorLabeler(self._match_threshold,
#                                          self._unmatched_threshold)
#    (cls_targets, box_targets, att_targets, cls_weights,
#     box_weights) = anchor_labeler.label_anchors(
#         anchor_boxes, boxes, tf.expand_dims(classes, axis=1), attributes)
#
#    # Casts input image to desired data type.
#    image = tf.cast(image, dtype=self._dtype)

#    # Packs labels for model_fn outputs.
#    labels = {
#        'cls_targets': cls_targets,
#        'box_targets': box_targets,
#        'anchor_boxes': anchor_boxes,
#        'cls_weights': cls_weights,
#        'box_weights': box_weights,
#        'image_info': image_info,
#    }
    return image, classes, boxes

In [12]:
#DEBUG
ds = ds.map(_parse_train_data)

Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.


Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.


In [ ]:
#DEBUG
for image, classes, boxes in ds.take(2):
    #print(image.shape)
    print(boxes)

In [ ]:
#DEBUG
#for source_id, image, height, width, groundtruth_classes, groundtruth_is_crowd, groundtruth_area, groundtruth_boxes in ds.take(2):
#    print(source_id, image, height, width, groundtruth_classes, groundtruth_is_crowd, groundtruth_area, groundtruth_boxes)
for data in ds.take(200):
    #print(data['source_id'])
    #print(data['image'].shape)
    print(data['width'])
    print(data['height'])
    print(data['groundtruth_boxes'])
    #print(data['groundtruth_classes'])

In [36]:
#DEBUG
train_dataset = task.build_inputs(train_data_cfg)
print(train_dataset)

valid_dataset = task.build_inputs(valid_data_cfg)
print(valid_dataset)

<PrefetchDataset shapes: ((512, 384, 384, 3), {cls_targets: OrderedDict([(3, (512, 48, 48, 9)), (4, (512, 24, 24, 9)), (5, (512, 12, 12, 9)), (6, (512, 6, 6, 9)), (7, (512, 3, 3, 9))]), box_targets: OrderedDict([(3, (512, 48, 48, 36)), (4, (512, 24, 24, 36)), (5, (512, 12, 12, 36)), (6, (512, 6, 6, 36)), (7, (512, 3, 3, 36))]), anchor_boxes: {3: (512, 48, 48, 36), 4: (512, 24, 24, 36), 5: (512, 12, 12, 36), 6: (512, 6, 6, 36), 7: (512, 3, 3, 36)}, cls_weights: (512, 27621), box_weights: (512, 27621), image_info: (512, 4, 2)}), types: (tf.bfloat16, {cls_targets: OrderedDict([(3, tf.int64), (4, tf.int64), (5, tf.int64), (6, tf.int64), (7, tf.int64)]), box_targets: OrderedDict([(3, tf.float32), (4, tf.float32), (5, tf.float32), (6, tf.float32), (7, tf.float32)]), anchor_boxes: {3: tf.float32, 4: tf.float32, 5: tf.float32, 6: tf.float32, 7: tf.float32}, cls_weights: tf.float32, box_weights: tf.float32, image_info: tf.float32})>
<PrefetchDataset shapes: ((512, 384, 384, 3), {cls_targets: Or

In [ ]:
#DEBUG
for image, metadata in train_dataset.take(2):
    #print(image.shape)
    #print(metadata.keys())
    #print(metadata['cls_targets']['3'].shape) # all -1 ?
    #print(metadata['box_targets']['3'][0][0][0]) # all the same ?
    #print(metadata['cls_weights']) # all zeros ?
    #print(metadata['image_info']) # this looks like boxes !
    # Conclusion: ALL WRONG

In [ ]:
# DEBUG
# model garden code copy-pasted for debugging...
parser_config = retinanet_cfg.Parser()
data_cfg = valid_data_cfg

decoder = modelgarden.vision.beta.dataloaders.tf_example_decoder.TfExampleDecoder()
parser = modelgarden.vision.beta.dataloaders.retinanet_input.Parser(
    output_size=params.task.model.input_size[:2],
    min_level=params.task.model.min_level,
    max_level=params.task.model.max_level,
    num_scales=params.task.model.anchor.num_scales,
    aspect_ratios=params.task.model.anchor.aspect_ratios,
    anchor_size=params.task.model.anchor.anchor_size,
    dtype=data_cfg.dtype,
    match_threshold=parser_config.match_threshold,
    unmatched_threshold=parser_config.unmatched_threshold,
    aug_rand_hflip=parser_config.aug_rand_hflip,
    aug_scale_min=parser_config.aug_scale_min,
    aug_scale_max=parser_config.aug_scale_max,
    skip_crowd_during_training=parser_config.skip_crowd_during_training,
    max_num_instances=parser_config.max_num_instances)

reader = modelgarden.vision.beta.dataloaders.input_reader_factory.input_reader_generator(
    data_cfg,
    dataset_fn=modelgarden.common.dataset_fn.pick_dataset_fn(data_cfg.file_type),
    decoder_fn=decoder.decode,
    #parser_fn=parser.parse_fn(data_cfg.is_training))
    parser_fn=parser.parse_fn(True))
dataset = reader.read(input_context=None)

print(dataset)

In [ ]:
# DEBUG
for image, labels in valid_dataset:
    print(image.shape)
    print('source_id', labels['groundtruths']['source_id'])
    print('height', labels['groundtruths']['height'])
    print('width', labels['groundtruths']['width'])
    print('num_detections', labels['groundtruths']['num_detections'])
    print('classes', labels['groundtruths']['classes'])
    print('cls_targets', labels['cls_targets'])
    
    print('boxes', labels['groundtruths']['boxes'])
    print('areas', labels['groundtruths']['areas'])
    print('is_crowds', labels['groundtruths']['is_crowds'])

In [9]:
print(MODEL_DIR)
model,_ = train_lib.run_experiment(
    distribution_strategy=strategy,
    task=task,
    mode="train_and_eval", # 'train', 'eval', 'train_and_eval' or 'continuous_eval'
    params=params,
    model_dir=MODEL_DIR)

gs://ml1-demo-martin/arthropod_jobs/job1625758868
Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.


Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.


restoring or initializing model...
initialized model.
train | step:      0 | training until step 360...
train | step:     45 | steps/sec:    0.1 | output: 
    {'box_loss': 0.009986863,
     'cls_loss': 0.9040905,
     'learning_rate': 0.016,
     'model_loss': 1.4034334,
     'total_loss': 1.4034334,
     'training_loss': 1.4034334}
saved checkpoint to gs://ml1-demo-martin/arthropod_jobs/job1625758868/ckpt-45.
train | step:     90 | steps/sec:    1.6 | output: 
    {'box_loss': 0.0055036703,
     'cls_loss': 0.63779324,
     'learning_rate': 0.016,
     'model_loss': 0.9129767,
     'total_loss': 0.9129767,
     'training_loss': 0.9129767}
train | step:    135 | steps/sec:    2.4 | output: 
    {'box_loss': 0.004524328,
     'cls_loss': 0.5752277,
     'learning_rate': 0.016,
     'model_loss': 0.801444,
     'total_loss': 0.801444,
     'training_loss': 0.801444}
train | step:    180 | steps/sec:    2.3 | output: 
    {'box_loss': 0.004102429,
     'cls_loss': 0.5308139,
     'learni

In [19]:
export_saved_model_lib.export_inference_graph(
      input_type='image_tensor',
      batch_size=4,
      input_image_size=OUTPUT_SIZE,
      params=params,
      checkpoint_path=MODEL_DIR,
      export_dir=MODEL_DIR,
      export_checkpoint_subdir='saved_chkpt',
      export_saved_model_subdir='saved_model')


FOR DEVS: If you are overwriting _tracking_metadata in your class, this property has been used to save metadata in the SavedModel. The metadta field will be deprecated soon, so please move the metadata to a different file.



FOR DEVS: If you are overwriting _tracking_metadata in your class, this property has been used to save metadata in the SavedModel. The metadta field will be deprecated soon, so please move the metadata to a different file.


INFO:tensorflow:Assets written to: gs://ml1-demo-martin/arthropod_jobs/job1625758868/saved_model/assets


INFO:tensorflow:Assets written to: gs://ml1-demo-martin/arthropod_jobs/job1625758868/saved_model/assets


## License
Copyright 2021 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.